In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:

%cd /content/gdrive/MyDrive
!git clone https://github.com/thangnch/MiAi_Traffic_Sign_Classify

/content/gdrive/MyDrive
Cloning into 'MiAi_Traffic_Sign_Classify'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 13 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (13/13), done.


In [ ]:
import os
import pandas as pd
#from scipy.misc import imread
from imageio import imread
import math
import numpy as np
import cv2
import keras
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from keras.models import Sequential
from keras.utils import to_categorical


In [ ]:
%cd /content/gdrive/MyDrive/MiAi_Traffic_Sign_Classify
!unzip GTSRB_Final_Training_Images.zip

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
  inflating: GTSRB/Final_Training/Images/00035/00000_00019.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00020.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00021.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00022.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00023.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00024.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00025.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00026.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00027.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00028.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00029.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00001_00000.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00001_00001.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00001_00002.ppm  
  inflating: GTSRB/Final_Training

In [ ]:
def load_data(input_size = (64,64), data_path =  '/content/gdrive/MyDrive/MiAi_Traffic_Sign_Classify/GTSRB/Final_Training/Images'):

    pixels = []
    labels = []
    # Loop qua các thư mục trong thư mục Images
    for dir in os.listdir(data_path):
        if dir == '.DS_Store':
            continue

        # Đọc file csv để lấy thông tin về ảnh
        class_dir = os.path.join(data_path, dir)
        info_file = pd.read_csv(os.path.join(class_dir, "GT-" + dir + '.csv'), sep=';')

        # Lăp trong file
        for row in info_file.iterrows():
            # Đọc ảnh
            pixel = imread(os.path.join(class_dir, row[1].Filename))
            # Trích phần ROI theo thông tin trong file csv
            pixel = pixel[row[1]['Roi.X1']:row[1]['Roi.X2'], row[1]['Roi.Y1']:row[1]['Roi.Y2'], :]
            # Resize về kích cỡ chuẩn
            img = cv2.resize(pixel, input_size)

            # Thêm vào list dữ liệu
            pixels.append(img)

            # Thêm nhãn cho ảnh
            labels.append(row[1].ClassId)

    return pixels, labels


In [ ]:
data_path = '/content/gdrive/MyDrive/MiAi_Traffic_Sign_Classify/GTSRB/Final_Training/Images'
pixels, labels = load_data(data_path=data_path)

KeyboardInterrupt: ignored

0

In [ ]:
def split_train_val_test_data(pixels, labels):
 
    # Chuẩn hoá dữ liệu pixels và labels
    pixels = np.array(pixels)
    labels = to_categorical(labels)
 
    # Nhào trộn dữ liệu ngẫu nhiên
    randomize = np.arange(len(pixels))
    np.random.shuffle(randomize)
    X = pixels[randomize]
    print("X=", X.shape)
    y = labels[randomize]
 
    # Chia dữ liệu theo tỷ lệ 60% train và 40% còn lại cho val và test
    train_size = int(X.shape[0] * 0.6)
    X_train, X_val = X[:train_size], X[train_size:]
    y_train, y_val = y[:train_size], y[train_size:]
 
    val_size = int(X_val.shape[0] * 0.5) # 50% của phần 40% bên trên
    X_val, X_test = X_val[:val_size], X_val[val_size:]
    y_val, y_test = y_val[:val_size], y_val[val_size:]
 
    return X_train, y_train, X_val, y_val, X_test, y_test


In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test = split_train_val_test_data(pixels, labels)

X= (39209, 64, 64, 3)


In [ ]:
def build_model(input_shape=(64,64,3), filter_size = (3,3), pool_size = (2, 2), output_size = 43):
    model = Sequential([
        Conv2D(16, filter_size, activation='relu', input_shape=input_shape, padding='same'),
        BatchNormalization(),
        Conv2D(16, filter_size, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=pool_size),
        Dropout(0.2),
        Conv2D(32, filter_size, activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(32, filter_size, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=pool_size),
        Dropout(0.2),
        Conv2D(64, filter_size, activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(64, filter_size, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=pool_size),
        Dropout(0.2),
        Flatten(),
        Dense(2048, activation='relu'),
        Dropout(0.3),
        Dense(1024, activation='relu'),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(output_size, activation='softmax')
    ])
 
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-4), metrics=['accuracy'])
    return model

In [ ]:
# Build model với kích thước đầu vào 64x64 và output là 43 classes
model = build_model(input_shape=(64,64,3), output_size=43)

# Train model
epochs = 50
batch_size = 64

model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                               validation_data=(X_val, y_val))

model.save("traffic_sign_model.h5")
#model = keras.models.load_model("traffic_sign_model.h5")

# Kiểm tra model với dữ liệu mới
print(model.evaluate(X_test, y_test))

368/368 [==============================] - 16s 44ms/step - loss: 0.0089 - accuracy: 0.9970 - val_loss: 0.0227 - val_accuracy: 0.9952
Epoch 42/50
368/368 [==============================] - 16s 44ms/step - loss: 0.0091 - accuracy: 0.9973 - val_loss: 0.0242 - val_accuracy: 0.9943
Epoch 43/50
368/368 [==============================] - 16s 44ms/step - loss: 0.0051 - accuracy: 0.9984 - val_loss: 0.0186 - val_accuracy: 0.9958
Epoch 44/50
368/368 [==============================] - 16s 44ms/step - loss: 0.0034 - accuracy: 0.9993 - val_loss: 0.0256 - val_accuracy: 0.9948
Epoch 45/50
368/368 [==============================] - 16s 44ms/step - loss: 0.0073 - accuracy: 0.9982 - val_loss: 0.0301 - val_accuracy: 0.9935
Epoch 46/50
368/368 [==============================] - 16s 44ms/step - loss: 0.0054 - accuracy: 0.9982 - val_loss: 0.0313 - val_accuracy: 0.9936
Epoch 47/50
368/368 [==============================] - 16s 43ms/step - loss: 0.0040 - accuracy: 0.9987 - val_loss: 0.0234 - val_accuracy: 0.99

In [ ]:
print(model.evaluate(X_test, y_test))

246/246 [==============================] - 2s 9ms/step - loss: 0.0275 - accuracy: 0.9943
[0.027462465688586235, 0.9942616820335388]


In [ ]:
from keras.models import load_model
import cv2
import numpy as np
from keras.preprocessing import image
class_names = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41','42']
model = load_model('/content/gdrive/MyDrive/MiAi_Traffic_Sign_Classify/traffic_sign_model.h5')
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
test_image = image.load_img('/content/gdrive/MyDrive/MiAi_Traffic_Sign_Classify/test1.png',target_size=(64,64,3))

test_image = image.img_to_array(test_image)
print(test_image.shape)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
print(result)
print(result.max())
max_index_row = np.argmax(result, axis=1)
print(max_index_row)

(64, 64, 3)
[[4.78069969e-15 6.38457065e-11 1.35798265e-11 1.98286564e-13
  3.56811192e-10 1.10917577e-11 1.62647708e-11 1.67913383e-12
  3.33257519e-11 1.37171592e-13 3.05153541e-11 7.17029547e-10
  5.76297303e-08 9.81698104e-05 1.78539925e-18 2.44884381e-13
  1.46926970e-16 1.94667069e-12 7.70921616e-14 2.32165039e-14
  1.33213865e-11 3.65663573e-15 4.45475579e-13 1.15198415e-11
  1.38291358e-11 1.74614114e-14 6.05170011e-11 2.14584730e-12
  3.30037113e-15 5.16450327e-13 1.94890317e-16 2.64866847e-14
  4.62590487e-11 1.14400781e-10 1.85389686e-02 1.08776699e-09
  5.26905808e-07 3.26069283e-09 9.81362164e-01 3.40813033e-12
  1.01399687e-10 2.30710451e-11 5.29955015e-15]]
0.98136216
[38]
